# Онлайн алгоритмы в задаче формирования портфеля ценных бумаг

Работа выполнена студентами группы М8О-411Б-19: \
Терешков Алексей \
Мамченков Дмитрий \
Алимов Исмаил

## Постановка задачи

В задаче формирования портфеля ценных бумаг допустимыми решениями являются всевозможные распределения состояния, имеющегося у игрока, т.е. элементы стандартного симплекса:

$$
\Delta_d = \left\{
x \in \mathbb{R}^d
:\;
\sum_{i = 1}^d x_i = 1,
\;
x_i \geq 0
\right\}.
$$

Противник (природа) независимо выбирает рыночную доходность активов,
т.е. вектор $r_t \in \mathbb{R}_+^d$,
у которого $i$-ая компонента определяется формулой

$$
[r_t]_i = \frac{\text{цена единицы $i$-го актива в момент } t + 1}{\text{цена единицы $i$-го актива в момент } t}.
$$

*Цель игрока* — максимизировать свое состояние по проишествии $T$ раундов, которая эквивалентна максимизации величины

$$
\sum_{t=1}^T \log r_t^\top x_t,
$$

которую будем называть *логарифмическим приростом* портфеля ценных бумаг к моменту времени $T$.

Регрет алгоритма $\mathcal{A}$ к раунду $T$ определяется формулой

$$
\mathrm{regret}_T(\mathcal{A})
=
\sum_{t=1}^T f_t(x_t)
-
\min_{x \in \Delta_d}
\sum_{t = 1}^T f_t(x),
$$

где $f_t(x) = -\log r_t^\top x$.

В этом задании вам предстоит реализовать алгоритмического инвестора на основе онлайн градиентного спуска, принимающего решения о перераспределении имеющихся средств между акциями российского фондового рынка:
- в момент времени $t$ алгоритм должен предложить распределение средств между активами $x_t$;
- после этого алгоритму открывается вектор рыночных доходностей $r_t$ из истории наблюдений;
- алгоритм должен использовать поступившую информацию для корректировки и принятия решения на следующей итерации.

## Подготовка данных (3 балла)

Загрузите котировки (сформируйте набор данных) акций `RASP`, `GAZP`, `DSKY`, `SBER`, `KMAZ`, `RUAL` c 01.04.2020 по 22.04.2022 с интервалом в один день (раунд игры = день). Выведите таблицу (`pd.DataFrame`), в которой столбцы содержат временные ряды для каждого тикера, а их названия — тикеры. **[1 балл]**

In [ ]:
import pandas as pd
import numpy as np
import random

import apimoex
import requests

import matplotlib.pyplot as plt
from numpy.typing import ArrayLike

In [ ]:
stock_names = ['RASP', 'GAZP', 'DSKY', 'SBER', 'KMAZ', 'RUAL']
start_date = '2020-04-01' 
end_date = '2022-04-22'

In [ ]:
def load_data(stock_names=stock_names, start=start_date, end=end_date):
    df = pd.DataFrame()
    with requests.Session() as session:
        for stock in stock_names:
            data = apimoex.get_board_history(session, stock, start=start, end=end, columns=['CLOSE', 'TRADEDATE'])
            df_stock = pd.DataFrame(data)
            df_stock.set_index('TRADEDATE', inplace=True)
            df[stock] = df_stock
    return df


def prepare_r(df_stocks):
    return (df_stocks.shift(-1) / df_stocks).dropna()

In [ ]:
data = load_data()
data_clear = data.dropna()
r = prepare_r(data_clear)

In [ ]:
data_clear

Визуализируйте поведение всех тикеров на всём горизонте игры. **[1 балл]**

In [ ]:
data_clear.plot()

Постройте таблицу (с тем же заголовком), содержащую по столбцам временные ряды *рыночных доходностей* тикеров. **[1 балл]**


In [ ]:
r

## Онлайн градиентный спуск (8 баллов)

Приведите выражение для градиента функции $f_t(x)$ и запишите явную формулу для итерации OGD. **[1 балл]**

Заменим операцию $ r_t^\top x_t$ на $ (\vec{r_t} \cdot \vec{x_t})$

В таком случае:
$$ f_t(x) = -\log (\vec{r} \cdot \vec{x}) $$

$$\nabla f_t(x) = -\frac{\vec{r}} {(\vec{r} \cdot \vec{x})} $$

Формула перехода:
$$ x_{t+1} = \Pi_{\mathcal{D}}(x_t - \alpha_t\nabla f_t(x_t)) $$ 
 


Реализуйте инвестора, принимающего решение на основе итераций онлайн градиентного спуска. **[4 балла]**

Используйте готовую функцию `simplex_projection`, проектирующую входной вектор на стандартный симплекс.

In [ ]:
def _unsafe_simplex_projection(s: ArrayLike, norm_constraint: float) -> ArrayLike:
    """Находит проекцию на симплекс."""
    u = np.sort(s)[::-1]
    cssv = np.cumsum(u)
    rho = np.nonzero(u * np.arange(1, len(u) + 1) > (cssv - norm_constraint))[0][-1]
    theta = (cssv[rho] - norm_constraint) / (rho + 1.0)
    return np.maximum(s - theta, 0)


def simplex_projection(s: ArrayLike):
    """Возвращает проекцию на единичный симплекс."""
    return s if np.sum(s) == 1 else _unsafe_simplex_projection(s, 1.0)

In [ ]:
def get_grad(r, x):
    return -r / np.dot(r, x)

In [ ]:
def descent_step(this_state, r, alpha, grad_func):
    x = this_state
    new_x = simplex_projection(x - alpha * grad_func(r, x))
    return new_x


def descent(begin_state, df_r, grad_func, lr_scheduler=lambda i: 0.1):
    x = begin_state

    n = df_r.shape[0]

    profit = []
    vectors = [x]

    profit = 1.0
    profit_vec = [profit]

    for i in range(n):
        r = df_r.iloc[i].values  # "после этого алгоритму открывается вектор рыночных доходностей rt из истории наблюдений"
        alpha = lr_scheduler(i)
        profit *= (r * x).sum()
        profit_vec.append(profit)
        new_x = descent_step(x, r, alpha, grad_func) # "алгоритм должен использовать поступившую информацию для корректировки и принятия решения на следующей итерации."
        x = new_x # в момент времени t алгоритм должен предложить распределение средств между активами xt 
        vectors.append(new_x)

    return (vectors, profit_vec)
    

### Запуск OGD

Как будете выбирать шаг при запуске OGD? Приведите исчерпывающее объяснение и мотивацию. **[1 балл]**

Длину шага выберем ситуативно, но вне зависимости от ситуации оставим его константной, так как ситуация на рынке меняется, а затухающая длина шага будет способствовать застыванию наших ответов, отчего на дистанции будет решать ситуация на рынке, которая была в начале.

Начиная с равномерного распределения состояния между активами, запустите OGD на всём горизонте игры. Визуализируйте динамику логарифмического прироста портфеля ценных бумаг за весь период для OGD и инвестора, зафиксировавшего своё (произвольное) распределение в начале игры. **[2 балла]**

### Подсчет прибыли для константного инвестора (без корректировки котировок)

In [ ]:
begin_x = np.ones(r.shape[1]) * (1/r.shape[1])

In [ ]:
vectors, naive_profit = descent(begin_x, r, get_grad, lr_scheduler=lambda x: 0)

In [ ]:
profit_to_date = pd.DataFrame(index=data_clear.index, columns=["profit"])
profit_to_date['profit'] = naive_profit

In [ ]:
graph = profit_to_date.plot()
graph.set_yscale('log')

Прибыльность константного инвестора:

In [ ]:
profit_to_date['profit'][-1]

### Подсчет прибыли с использованием алгоритма OGD (с поиском оптимальной константы для длины шага)

In [ ]:
alpha_array = np.logspace(1, 10, 1000, base=5.1) * 0.0001

In [ ]:
max_profit = [0]
best_alpha = -1.0

for i in alpha_array:
    vectors, profit = descent(begin_x, r, get_grad, lr_scheduler=lambda x: i)
    if profit[-1] > max_profit[-1]:
      max_profit = profit
      best_alpha = i

In [ ]:
best_alpha

Для данного метода лучшей константой alpha оказалась 0.436358

In [ ]:
profit_to_date = pd.DataFrame(index=data_clear.index, columns=["profit"])
profit_to_date['profit'] = max_profit

In [ ]:
graph = profit_to_date.plot()
graph.set_yscale('log')

Прибыльность (с использованием OGD):

In [ ]:
profit_to_date['profit'][-1]

## Дополнительное задание (4 балла)

Реализуйте любой подходящий канонический онлайн алгоритм из курса или даже их композицию. Визуализируйте динамику логарифмического прироста портфеля ценных бумаг за весь период для OGD и инвестора, принимающего решения на основе выбранного алгоритма. Сравните итоговые распределения, полученные с помощью этих алгоритмов, и попробуйте сделать выводы.

### Онлайн метод Ньютона (ONS)

Онлайн метод Ньютона устроен следующим образом:

На начале пределяются параметры $ \gamma, \varepsilon > 0,  $ и создается матрица $\Alpha = \varepsilon \Iota_{d \times d}$

Шаг алгоритма:
$$\Alpha_{t+1} = \Alpha_{t} + \nabla f_t(x_t) \nabla f_t(x_t)^\top $$
$$x_{t+1} = \Pi_{\mathcal{D}, A_t}(x_{t} - \frac{1}{\gamma} A_t^{-1} \nabla f_t(x_t) )$$
, где $\Pi_{\mathcal{D}, A_t}(y)$ - оператор проектирования на множество $ \mathcal{D}$ 
$$ \Pi_{\mathcal{D}, A_t}(y) = \arg\min_{x \in \mathcal{D}} \bigl\{   \sqrt{(x-y)^\top A_t (x-y)} \bigr \} $$


In [136]:
from cvxpy import *

def newton_proj(A, y):
    x = cvxpy.Variable(A.shape[0])
    problem = Problem(
        Minimize(quad_form(x, A)),
                 [(x - y)>=0, cvxpy.sum(x - y) == 1]
                 )
    problem.solve()
    return x.value - y

def newton_step(this_state, r, A, gamma, grad_func):
    x = this_state
    grad = grad_func(r, x)
    exp = x - np.linalg.inv(A) @ grad / gamma
    new_x = newton_proj(A, exp)
    new_A = A + grad @ grad.T
    return new_x, new_A


def newton_method(begin_state, df_r, grad_func, eps, gamma):
    x = begin_state
    A = eps * np.eye(df_r.shape[1], df_r.shape[1])
    n = df_r.shape[0]

    profit = []
    vectors = [x]

    profit = 1.0
    profit_vec = [profit]

    for i in range(n):
        r = df_r.iloc[i].values  # "после этого алгоритму открывается вектор рыночных доходностей rt из истории наблюдений"
        print(r, x, (r * x).sum(), np.dot(r, x))
        profit *= (r * x).sum()
        profit_vec.append(profit)
        # print(grad_func(r, x))
        new_x, new_A = newton_step(x, r, A, gamma, grad_func) # "алгоритм должен использовать поступившую информацию для корректировки и принятия решения на следующей итерации."
        x = new_x # в момент времени t алгоритм должен предложить распределение средств между активами xt 
        A = new_A
        vectors.append(new_x)

    return (vectors, profit_vec)


In [ ]:
begin_x = np.ones(r.shape[1]) * (1/r.shape[1])



vectors, naive_profit = newton_method(begin_x, r, get_grad, 0.0001, 0.0001)

profit_to_date = pd.DataFrame(index=data_clear.index, columns=["profit"])
profit_to_date['profit'] = naive_profit

graph = profit_to_date.plot()
graph.set_yscale('log')

profit_to_date['profit'][-1]

In [ ]:
# alpha_array_gamma = np.logspace(1, 10, 5, base=5.1) * 0.0001
# alpha_array_eps = np.logspace(1, 10, 5, base=5.1) * 0.0001

# max_profit = [0]
# best_gamma = -1.0
# best_eps = -1.0


# for gamma in alpha_array_gamma:
#     for eps in alpha_array_eps:
#       print(gamma, eps)
#       vectors, profit = newton_method(begin_x, r, get_grad, eps, gamma)
#       if profit[-1] > max_profit[-1]:
#         max_profit = profit
#         best_gamma = gamma
#         best_eps = eps

In [ ]:
max_profit

# Вывод: 

Наивный алгоритм (если взять равномерно все активы и не пересобирать портфель) выдает прибыльность 1.7. Алгоритм OGD выдает прибыльность 2.14.

Алгоритм OGD приносит прибыли не сильно выше роста рынка, однако ситуация на рынке не предсказывается очевидным образом, и даже такой результат - уже отлично.

